In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/temporal-fusion-transformer/batch_14.parquet
/kaggle/input/temporal-fusion-transformer/batch_21.parquet
/kaggle/input/temporal-fusion-transformer/batch_20.parquet
/kaggle/input/temporal-fusion-transformer/batch_03.parquet
/kaggle/input/temporal-fusion-transformer/batch_07.parquet
/kaggle/input/temporal-fusion-transformer/batch_18.parquet
/kaggle/input/temporal-fusion-transformer/batch_12.parquet
/kaggle/input/temporal-fusion-transformer/batch_11.parquet
/kaggle/input/temporal-fusion-transformer/batch_10.parquet
/kaggle/input/temporal-fusion-transformer/batch_08.parquet
/kaggle/input/temporal-fusion-transformer/batch_09.parquet
/kaggle/input/temporal-fusion-transformer/batch_06.parquet
/kaggle/input/temporal-fusion-transformer/batch_15.parquet
/kaggle/input/temporal-fusion-transformer/batch_17.parquet
/kaggle/input/temporal-fusion-transformer/batch_02.parquet
/kaggle/input/temporal-fusion-transformer/batch_05.parquet
/kaggle/input/temporal-fusion-transformer/batch_16.parqu

In [3]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pickle
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Lớp TimeSeriesDataset để tạo các chuỗi dữ liệu thời gian
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_len=432, pred_len=144):
        self.input_len = input_len  # Độ dài chuỗi đầu vào (432 × 10 phút = 72 giờ)
        self.pred_len = pred_len    # Độ dài chuỗi dự báo (144 × 10 phút = 24 giờ)
        self.sequences = self.create_sequences(data)
    
    def create_sequences(self, data):
        sequences = []
        total_len = self.input_len + self.pred_len
        for i in range(len(data) - total_len + 1):
            input_seq = data[i:i + self.input_len]
            target_seq = data[i + self.input_len:i + total_len]
            sequences.append((input_seq, target_seq))
        return sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        x, y = self.sequences[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Lớp mô hình PatchTST
class PatchTST(nn.Module):
    def __init__(self, input_dim, patch_len=16, stride=8, d_model=64, num_layers=3, pred_len=144):
        super().__init__()
        self.patch_len = patch_len
        self.stride = stride
        self.pred_len = pred_len
        self.input_dim = input_dim
        self.linear = nn.Linear(patch_len * input_dim, d_model)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=4, batch_first=True),
            num_layers=num_layers
        )
        self.head = nn.Linear(d_model, input_dim * pred_len)
    
    def forward(self, x):  # x: [B, T, C]
        B, T, C = x.shape
        patches = x.unfold(1, self.patch_len, self.stride)  # [B, num_patches, patch_len, C]
        patches = patches.permute(0, 1, 3, 2).reshape(B, -1, self.patch_len * C)
        x = self.linear(patches)  # [B, num_patches, d_model]
        x = self.encoder(x)
        x = self.head(x[:, -1])  # [B, input_dim * pred_len]
        return x.reshape(B, self.pred_len, C)  # [B, pred_len, input_dim]

# Hàm huấn luyện mô hình
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Hàm đánh giá mô hình
def evaluate_model(model, dataloader, criterion, device, scaler=None):
    model.eval()
    mse_list = []
    total_loss = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            total_loss += loss.item()
            y_pred_np = y_pred.cpu().numpy()
            y_true_np = y.cpu().numpy()
            if scaler:
                y_pred_np = scaler.inverse_transform(y_pred_np.reshape(-1, y_pred_np.shape[-1])).reshape(y_pred_np.shape)
                y_true_np = scaler.inverse_transform(y_true_np.reshape(-1, y_true_np.shape[-1])).reshape(y_true_np.shape)
            mse = mean_squared_error(y_true_np.flatten(), y_pred_np.flatten())
            mse_list.append(mse)
    return total_loss / len(dataloader), np.mean(mse_list)

# Hàm huấn luyện và lưu mô hình cho 100 Square id ngẫu nhiên
def train_and_save_models(batch_folder, output_dir="models", input_len=432, pred_len=144, num_epochs=10, num_ids=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    features = ['SMS-in activity', 'SMS-out activity', 'Call-in activity', 'Call-out activity', 'Internet traffic activity']
    
    # Lấy danh sách file batch
    batch_files = [f for f in sorted(os.listdir(batch_folder)) if f.endswith(".parquet")]
    if not batch_files:
        print("Không tìm thấy file parquet trong thư mục")
        return
    
    # Chọn 100 Square id ngẫu nhiên từ batch đầu tiên
    first_batch_path = os.path.join(batch_folder, batch_files[0])
    print(f"Đang đọc batch đầu tiên: {batch_files[0]}")
    first_df = pd.read_parquet(first_batch_path)
    unique_square_ids = first_df['Square id'].unique()
    if len(unique_square_ids) < num_ids:
        print(f"Chỉ có {len(unique_square_ids)} Square id, chọn tất cả")
        selected_square_ids = unique_square_ids
    else:
        selected_square_ids = np.random.choice(unique_square_ids, size=num_ids, replace=False)
    
    print(f"Đã chọn {len(selected_square_ids)} Square id để huấn luyện")
    
    # Gộp dữ liệu từ tất cả batch cho các Square id được chọn
    all_data = []
    for batch_file in batch_files:
        batch_path = os.path.join(batch_folder, batch_file)
        print(f"Đang đọc {batch_file}")
        df = pd.read_parquet(batch_path)
        df['Time Interval'] = pd.to_datetime(df['Time Interval'], unit='ms')
        # Lọc chỉ các Square id được chọn
        df = df[df['Square id'].isin(selected_square_ids)]
        all_data.append(df)
    
    # Gộp tất cả batch thành một DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values(['Square id', 'Time Interval'])
    
    # Huấn luyện mô hình cho từng Square id
    for square_id in selected_square_ids:
        print(f"\n=== Huấn luyện cho Square id {square_id} ===")
        group = combined_df[combined_df['Square id'] == square_id].sort_values('Time Interval')
        data = group[features].values
        
        # Chuẩn hóa dữ liệu
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data)
        
        # Lấy 432 bước thời gian gần nhất (hoặc toàn bộ nếu ít hơn)
        history_data = data[-input_len:]  # Dữ liệu gốc (chưa chuẩn hóa)
        history_times = group['Time Interval'].values[-input_len:]  # Thời gian tương ứng
        
        # Tạo dataset
        dataset = TimeSeriesDataset(data_scaled, input_len, pred_len)
        if len(dataset) < 10:
            print(f"Bỏ qua Square id {square_id}: không đủ dữ liệu ({len(dataset)} chuỗi)")
            continue
        
        # Chia tập huấn luyện và kiểm tra
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        
        # Khởi tạo mô hình
        model = PatchTST(input_dim=len(features), pred_len=pred_len).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        
        # Vòng lặp huấn luyện với dừng sớm
        best_val_loss = float('inf')
        patience = 3
        patience_counter = 0
        
        for epoch in range(num_epochs):
            train_loss = train_model(model, train_loader, optimizer, criterion, device)
            val_loss, val_mse = evaluate_model(model, val_loader, criterion, device, scaler)
            print(f"[Square id: {square_id}, Epoch {epoch+1}] Mất mát huấn luyện: {train_loss:.4f}, Mất mát kiểm tra: {val_loss:.4f}, MSE kiểm tra: {val_mse:.4f}")
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Lưu mô hình
                model_path = os.path.join(output_dir, f"patchtst_square_{square_id}.pth")
                torch.save(model.state_dict(), model_path)
                # Lưu scaler và dữ liệu lịch sử
                save_data = {
                    'scaler': scaler,
                    'history_data': history_data,  # Dữ liệu gốc
                    'history_times': history_times  # Mốc thời gian
                }
                scaler_history_path = os.path.join(output_dir, f"scaler_history_square_{square_id}.pkl")
                with open(scaler_history_path, 'wb') as f:
                    pickle.dump(save_data, f)
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Dừng sớm tại epoch {epoch+1} cho Square id {square_id}")
                    break

# Hàm dự báo
def predict(square_id, hour, day_of_week, model_dir="models", input_len=432, pred_len=144):
    # Ánh xạ ngày trong tuần sang chỉ số
    days = ['thứ hai', 'thứ ba', 'thứ tư', 'thứ năm', 'thứ sáu', 'thứ bảy', 'chủ nhật']
    try:
        day_idx = days.index(day_of_week.lower())
    except ValueError:
        return f"Ngày trong tuần không hợp lệ: {day_of_week}. Phải là một trong {days}"
    
    if not (0 <= hour <= 23):
        return f"Giờ không hợp lệ: {hour}. Phải nằm trong khoảng 0-23"
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    features = ['SMS-in activity', 'SMS-out activity', 'Call-in activity', 'Call-out activity', 'Internet traffic activity']
    
    # Tải mô hình và dữ liệu lịch sử
    model_path = os.path.join(model_dir, f"patchtst_square_{square_id}.pth")
    scaler_history_path = os.path.join(model_dir, f"scaler_history_square_{square_id}.pkl")
    
    if not os.path.exists(model_path) or not os.path.exists(scaler_history_path):
        return f"Không tìm thấy mô hình hoặc dữ liệu cho Square id {square_id}"
    
    model = PatchTST(input_dim=len(features), pred_len=pred_len).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    with open(scaler_history_path, 'rb') as f:
        saved_data = pickle.load(f)
    scaler = saved_data['scaler']
    history_data = saved_data['history_data']
    history_times = pd.to_datetime(saved_data['history_times'])
    
    # Kiểm tra dữ liệu lịch sử
    if len(history_data) < input_len:
        return f"Không đủ dữ liệu lịch sử cho Square id {square_id} (chỉ có {len(history_data)} bước, cần {input_len})"
    
    # Tạo DataFrame lịch sử để lọc theo ngày và giờ
    history_df = pd.DataFrame(history_data, columns=features)
    history_df['Time Interval'] = history_times
    history_df['Day of Week'] = history_df['Time Interval'].dt.dayofweek
    history_df['Hour'] = history_df['Time Interval'].dt.hour
    
    # Tìm chuỗi gần nhất kết thúc tại giờ và ngày trong tuần
    target_time = history_df[(history_df['Day of Week'] == day_idx) & (history_df['Hour'] == hour)]
    if target_time.empty:
        # Fallback: Lấy 432 bước gần nhất bất kể ngày/giờ
        input_data = history_df.tail(input_len)
    else:
        latest_time = target_time['Time Interval'].max()
        input_data = history_df[history_df['Time Interval'] <= latest_time].tail(input_len)
    
    if len(input_data) < input_len:
        return f"Không đủ dữ liệu lịch sử sau lọc cho Square id {square_id}"
    
    # Chuẩn bị dữ liệu đầu vào
    data = input_data[features].values
    data = scaler.transform(data)
    x = torch.tensor(data, dtype=torch.float32).unsqueeze(0).to(device)  # [1, input_len, input_dim]
    
    # Dự báo
    with torch.no_grad():
        y_pred = model(x).cpu().numpy()  # [1, pred_len, input_dim]
    y_pred = scaler.inverse_transform(y_pred[0]).reshape(pred_len, len(features))  # [pred_len, input_dim]
    
    # Tạo mốc thời gian cho dự báo (dựa trên thời gian cuối cùng trong lịch sử)
    latest_time = input_data['Time Interval'].iloc[-1]
    pred_times = [latest_time + timedelta(minutes=10 * i) for i in range(1, pred_len + 1)]
    
    # Định dạng kết quả
    result = []
    for i, t in enumerate(pred_times):
        row = [t.strftime('%H:%M')]
        for j, feature in enumerate(features):
            row.append(f"{y_pred[i, j]:.2f}")
        result.append(row)
    
    columns = ['Thời gian'] + features
    result_df = pd.DataFrame(result, columns=columns)
    
    return result_df

# Giao diện dòng lệnh
def main():
    batch_folder = "/kaggle/input/temporal-fusion-transformer"
    model_dir = "models"
    
    # Huấn luyện mô hình nếu chưa có
    if not os.path.exists(model_dir) or not any(f.endswith('.pth') for f in os.listdir(model_dir)):
        print("Đang huấn luyện các mô hình cho 100 Square id ngẫu nhiên...")
        train_and_save_models(batch_folder, model_dir)
    
    # Tương tác dự báo
    while True:
        print("\nNhập thông tin dự báo (hoặc 'thoát' để kết thúc):")
        square_id = input("Square id: ")
        if square_id.lower() == 'thoát':
            break
        try:
            square_id = int(square_id)
        except ValueError:
            print("Square id phải là số nguyên")
            continue
        
        hour = input("Giờ (0-23): ")
        try:
            hour = int(hour)
        except ValueError:
            print("Giờ phải là số nguyên")
            continue
        
        day_of_week = input("Ngày trong tuần (VD: Thứ hai): ")
        
        result = predict(square_id, hour, day_of_week, model_dir)
        if isinstance(result, str):
            print(result)
        else:
            print("\nDự báo cho 24 giờ tiếp theo:")
            print(result.to_string(index=False))

if __name__ == "__main__":
    main()

Đang huấn luyện các mô hình cho 100 Square id ngẫu nhiên...
Đang đọc batch đầu tiên: batch_01.parquet
Đã chọn 10 Square id để huấn luyện
Đang đọc batch_01.parquet
Đang đọc batch_02.parquet
Đang đọc batch_03.parquet
Đang đọc batch_04.parquet
Đang đọc batch_05.parquet
Đang đọc batch_06.parquet
Đang đọc batch_07.parquet
Đang đọc batch_08.parquet
Đang đọc batch_09.parquet
Đang đọc batch_10.parquet
Đang đọc batch_11.parquet
Đang đọc batch_12.parquet
Đang đọc batch_13.parquet
Đang đọc batch_14.parquet
Đang đọc batch_15.parquet
Đang đọc batch_16.parquet
Đang đọc batch_17.parquet
Đang đọc batch_18.parquet
Đang đọc batch_19.parquet
Đang đọc batch_20.parquet
Đang đọc batch_21.parquet

=== Huấn luyện cho Square id 6059 ===
[Square id: 6059, Epoch 1] Mất mát huấn luyện: 0.6941, Mất mát kiểm tra: 0.5769, MSE kiểm tra: 18643.4297
[Square id: 6059, Epoch 2] Mất mát huấn luyện: 0.5823, Mất mát kiểm tra: 0.5443, MSE kiểm tra: 17984.5293
[Square id: 6059, Epoch 3] Mất mát huấn luyện: 0.5464, Mất mát kiể

Square id:  3804
Giờ (0-23):  20
Ngày trong tuần (VD: Thứ hai):  thứ hai



Dự báo cho 24 giờ tiếp theo:
Thời gian SMS-in activity SMS-out activity Call-in activity Call-out activity Internet traffic activity
    23:00           -0.07            -0.27            -0.10             -0.04                      0.79
    23:10            0.09             0.03             0.27              0.24                     16.96
    23:20            0.29             0.22             0.14              0.21                      4.19
    23:30           -0.36            -0.28             0.11              0.01                     13.92
    23:40            0.71             0.40             0.29              0.39                      6.31
    23:50           -0.00            -0.03             0.06              0.07                     13.94
    00:00            0.51             0.18             0.18              0.19                      5.98
    00:10            0.28             0.26             0.25              0.22                     16.27
    00:20            0.06         

Square id:  thoát
